In [57]:
#백테스팅 결과 참고 후, 수정 필요
#유니버스 - 소형주
#세 가지 팩터만 활용하기로 함

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pykrx import stock

def perform_screening(end_date):
    end_date_datetime = datetime.strptime(end_date, "%Y%m%d")
    start_date_datetime = end_date_datetime - timedelta(days=30)
    start_date = start_date_datetime.strftime("%Y%m%d")

    # 시가총액 데이터 가져오기
    
    kospi_small = stock.get_index_portfolio_deposit_file("1004")

    market_cap_data = stock.get_market_cap(end_date, market="ALL")
    market_cap_data = market_cap_data[['종가', '시가총액']]

    filtered_market_cap_data = market_cap_data.loc[kospi_small]
    
    gains = []
    filtered_tickers = filtered_market_cap_data.index

    for ticker in filtered_tickers:
        df = stock.get_market_ohlcv(start_date, end_date, ticker, "m")
        gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
        gains.append(gain)

    result_df = pd.DataFrame({'ticker': filtered_tickers, 'gain': gains})
    result_df.set_index('ticker', inplace=True)
    gain_25 = result_df['gain'].quantile(0.25)
    selected_rows = result_df['gain'] < gain_25
    temp_df = result_df[selected_rows]
    selected_tickers = temp_df.index.tolist()

    # 평균값 계산
    revisions = []

    for ticker in selected_tickers:
        data = stock.get_market_ohlcv(start_date, end_date, ticker, "d")
        mean_price = data['종가'].mean()
        close_temp = stock.get_market_ohlcv(start_date, end_date, ticker, "d")
        close = close_temp.iloc[0,3]
        revision = (close / mean_price) * 100
        revisions.append(revision)

    result_df = pd.DataFrame({'ticker': selected_tickers, 'revision': revisions})
    result_df.set_index('ticker', inplace=True)
    revision_25 = result_df['revision'].quantile(0.25)
    selected_rows = result_df['revision'] < revision_25
    revision_df = result_df[selected_rows]

    # 평균값 계산
    mean_values = []

    for ticker in revision_df.index:
        data1 = stock.get_market_trading_value_by_date(start_date, end_date, ticker)
        data2 = stock.get_market_cap(start_date, end_date, ticker)['시가총액']
        data1['개인순매수'] = (data1['개인'] / data2) * 100
        mean = data1['개인순매수'].mean()
        mean_values.append(mean)

    final_df = pd.DataFrame({'ticker': revision_df.index, 'mean': mean_values})
    final_df.set_index('ticker', inplace=True)
    final_75 = final_df['mean'].quantile(0.75)
    selected_rows = final_df['mean'] > final_75
    screening = final_df[selected_rows]

    return screening

# 스크리닝 함수 호출
end_date = "20240322"
screened_stocks = perform_screening(end_date)
print(screened_stocks.index)

Index(['007810', '010690', '001250', '163560', '011300', '004770', '011420',
       '033250'],
      dtype='object', name='ticker')


In [51]:
# 백테스트 코드
# 간단하게 CAGR과 누적 수익률을 알아보겠습니다

In [52]:
# 인덱스들 종가 불러오기
final_values = []
for ticker in screened_stocks.index:
    data = stock.get_market_ohlcv("20240322", "20240322", ticker, "d")
    final = data.iloc[0,3]
    final_values.append(final)

final_df = pd.DataFrame({'ticker': screened_stocks.index, '종가': final_values})
cash = 100000000 # 모의투자 시작금액인 1억으로 시작하겠습니다.
money = cash

if len(screened_stocks.index)==0:
    allocation=0
else:
    allocation = money / len(screened_stocks.index) # 동일 비중이기 때문에 보유 현금 / 투자할 종목수 로 나눠줍니다.

final_df['매수 수량'] = allocation / final_df['종가']
final_df

,ticker,종가,매수 수량
0,007810,18100,690.607735
1,010690,11460,1090.750436
2,001250,2760,4528.985507
3,163560,8300,1506.024096
4,011300,1238,10096.930533
5,004770,2150,5813.953488
6,011420,2505,4990.019960
7,033250,1365,9157.509158


In [53]:
import FinanceDataReader as fdr #벤치마킹 지수

ref = fdr.DataReader(
    symbol = 'KS11',
    start = '20190319',
    end = '20240322')

num_of_year = int(len(ref.index)/365) #90일을 잡았습니다
ref_cagr = ((ref['Close'].iloc[-1] / ref['Close'].iloc[0])**(1/num_of_year)) -1
ref_cagr*100

ref['agr'] = (ref['Close'] / ref['Open'])
ref['cagr'] = ((ref['Close'] / ref['Open']))**(1/num_of_year) -1

tickers = screened_stocks.index
date_range = pd.date_range(start="20190319", end="20240322")
result_df = pd.DataFrame(index=date_range)

for ticker in tickers:
    data = stock.get_market_ohlcv("20190319", "20240322", ticker, "d")
    data['수익률'] = (data['종가'] / data['시가'])
    result_df[ticker] = data['수익률']

# NaN 값을 이전 행의 값으로 채우기
result_df.fillna(method='ffill', inplace=True)

result_df.index.name = '날짜'

# DataFrame에서 무한대 값을 NaN으로 대체
result_df.replace([np.inf, -np.inf], np.nan, inplace=True)
result_df['포트폴리오 수익률'] = result_df.mean(axis=1, skipna=True)
result_df['CAGR'] = (data['수익률'] ** (1 / num_of_year) - 1) * 100
result_df

,007810,010690,001250,163560,011300,004770,011420,033250,포트폴리오 수익률,CAGR
날짜,,,,,,,,,,
2019-03-19,0.993579,1.006173,0.994555,0.996937,0.989943,1.027634,0.941718,1.022947,0.996686,0.759118
2019-03-20,0.976000,1.014228,0.994505,0.991523,0.969828,0.978261,0.977199,0.988067,0.986201,-0.399365
2019-03-21,0.998387,1.054326,0.994475,1.007704,1.001484,0.991453,0.976667,0.985490,1.001248,-0.486034
2019-03-22,0.984051,0.980989,0.981550,0.985660,1.014749,0.982759,1.010067,1.014742,0.994321,0.489005
2019-03-23,0.984051,0.980989,0.981550,0.985660,1.014749,0.982759,1.010067,1.014742,0.994321,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-03-18,1.010811,0.990733,0.998205,0.987055,0.930855,1.009281,1.011742,1.004831,0.992939,0.160772
2024-03-19,0.985999,0.978741,0.983725,0.970166,1.040670,0.997680,0.966990,0.973811,0.987223,-0.880695
2024-03-20,1.023446,0.989601,1.003663,0.985311,0.970229,0.995349,1.008081,0.993631,0.996164,-0.212767


In [54]:
mean_return = result_df['포트폴리오 수익률'].mean()
(mean_return - 1) * 100

0.09687222210614799

In [55]:
import plotly.graph_objs as go
import plotly.io as pio

pio.templates["seaborn"] = go.layout.Template(layout=go.Layout(template="seaborn"))
pio.templates.default = "seaborn"

scatter_plot = go.Scatter(x=ref.index, y=ref['cagr'], mode='lines', name='코스피 CAGR')
scatter_plot_cagr = go.Scatter(x=result_df.index, y=result_df['CAGR'], mode='lines', name='포트폴리오 CAGR')

layout = go.Layout(title='<b>5년 백테스팅<b>',
                   xaxis=dict(title='날짜'),
                   yaxis=dict(title='CAGR'))

# 그래프 객체를 figure로 생성합니다.
fig = go.Figure(data=[scatter_plot, scatter_plot_cagr], layout=layout)

# 그래프 출력
fig.show()

In [39]:
import plotly.graph_objs as go
import plotly.io as pio

pio.templates["seaborn"] = go.layout.Template(layout=go.Layout(template="seaborn"))
pio.templates.default = "seaborn"

scatter_plot = go.Scatter(x=ref.index, y=ref['agr'], mode='lines', name='코스피 수익률')
scatter_plot_mean = go.Scatter(x=result_df.index, y=result_df['포트폴리오 수익률'], mode='lines', name='포트폴리오 수익률')

layout = go.Layout(title='<b>5년 백테스팅<b>',
                   xaxis=dict(title='날짜'),
                   yaxis=dict(title='수익률'))

# 그래프 객체를 figure로 생성합니다.
fig = go.Figure(data=[scatter_plot, scatter_plot_mean], layout=layout)

# 그래프 출력
fig.show()